# run51.ipynb

This notebook should be run within the fifty_one virtual environment.

Activate using:
````
source .venv/bin/activate
````

## References

- https://docs.voxel51.com/user_guide/evaluation.html#evaluating-models
- https://docs.voxel51.com/user_guide/plots.html#confusion-matrices

In [19]:
import fiftyone as fo
from fiftyone import ViewField as F
import fiftyone.brain as fob

from ultralytics import YOLO

In [20]:
ROOT_DIR = '/home/aubrey/Desktop/crb-damage-detection-models'
DATASET_DIR = f'{ROOT_DIR}/datasets/CRB005'
MODEL_DIR = f'{ROOT_DIR}/models/CRB005/weights/CRB005.pt'
FO_DATASET = 'fo_test_dataset_2'

# FUNCTIONS

In [21]:
def wake_mongo():

    # List all local datasets by name

    # Reports error 26 on first try, indicating a problem opening the Mongo database.
    # The following hack runs the code a second time. Seems to work well.
    # Note: A dataset can be deleted using fo.delete_dataset("dataset name")
    try:
        dataset_list = fo.list_datasets()
    except Exception as e:
        print(f'ERROR: {e}')
        print("Retrying...")
        dataset_list = fo.list_datasets()
    print(f'Local FiftyOne datasets: {dataset_list}')

In [22]:
def set_custom_color_scheme(dataset):
    """ 
    set custom color scheme for object classes
    """
    dataset.app_config.color_scheme = fo.ColorScheme(
        color_by="value",
        color_pool=[
            "#ff0000",  # red
            "#00ff00",  # green
            "#0000ff",  # blue
            "pink",
            "yellowgreen",
        ],
        fields=[
            {
                "path": "ground_truth",
                "fieldColor": "#ff00ff",  # magenta for the field
                "colorByAttribute": "label",
                "valueColors": [
                    {"value": "healthy", "color": "green"},
                    {"value": "damaged", "color": "yellow"},
                    {"value": "dead",    "color": "blue"},
                    {"value": "vut",     "color": "red"}
                ]
            },
            {
                "path": "predictions",
                "fieldColor": "#ff00ff",  # magenta for the field
                "colorByAttribute": "label",
                "valueColors": [
                    {"value": "healthy", "color": "green"},
                    {"value": "damaged", "color": "yellow"},
                    {"value": "dead",    "color": "blue"},
                    {"value": "vut",     "color": "red"}
                ]
            }
        ],
    )

# MAIN

In [23]:
wake_mongo()

if FO_DATASET in fo.list_datasets():
    dataset = fo.load_dataset(FO_DATASET)
    print(f'Loaded existing FiftyOne dataset: {FO_DATASET}')
else:
    print(f'Creating new FiftyOne dataset: {FO_DATASET}')
    
    # Create a new FiftyOne dataset named FO_DATASET from YOLOv5 formatted data
    dataset = fo.Dataset(FO_DATASET, persistent=True)
    for split in ['train', 'val']:
        dataset.add_dir(dataset_dir=DATASET_DIR, dataset_type=fo.types.YOLOv5Dataset, split=split, tags=[split])
        
    # Add detections by running the model on the dataset
    print(f'--- Adding detections using YOLOv11 model:\n{MODEL_DIR}')  
    # model = YOLO(MODEL_DIR)
    dataset.apply_model(model=YOLO(MODEL_DIR), progress=True)
    # print summary info about the dataset
    print(dataset)
    print(f'\ntags: {dataset.distinct("tags")}\n\n')
    
    # Evaluate the detections using ground truth labels
    eval_results = dataset.evaluate_detections(pred_field='predictions', gt_field='ground_truth', eval_key='eval')
    
    # Print evaluation report sorted by class frequency
    counts = dataset.count_values("ground_truth.detections.label")
    classes = sorted(counts, key=counts.get, reverse=True)
    eval_results.print_report(classes=classes)

    # Find and rank potential mistakes using the FiftyOne Brain
    fob.compute_mistakenness(dataset, "predictions", label_field="ground_truth")

    # Create a view to find the most "mistaken" samples
    # Mistakenness is a score that indicates how likely an annotation is wrong
    view = (
        dataset.sort_by("mistakenness", reverse=True)
        .limit(100) # Get the top 100
    )
    try:
        dataset.save_view('mistakenness-view', view)
    except:
        pass
    mistakenness_view = dataset.load_saved_view('mistakenness-view')
        
    # set custom color scheme for object classes    
    set_custom_color_scheme(dataset)
    
# show info about the dataset
print(dataset)

# Launch the FiftyOne app  
session = fo.launch_app(dataset, auto=False)

Local FiftyOne datasets: ['fo_test_dataset', 'fo_test_dataset_2', 'quickstart']
Loaded existing FiftyOne dataset: fo_test_dataset_2
Name:        fo_test_dataset_2
Media type:  image
Num samples: 151
Persistent:  True
Tags:        []
Sample fields:
    id:                fiftyone.core.fields.ObjectIdField
    filepath:          fiftyone.core.fields.StringField
    tags:              fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:        fiftyone.core.fields.DateTimeField
    last_modified_at:  fiftyone.core.fields.DateTimeField
    ground_truth:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:           fiftyone.core.fields.IntField
    eval_fp:           fiftyone.core.fields.IntField
    eval_fn:           fif

## At this point, the FiftyOne app should be available in a browser at http://localhost:5151

Trying to fix problem: object class list not in the app's lefthand panel

In [13]:
# Get the default active fields
active_fields = fo.DatasetAppConfig.default_active_fields(dataset)
# Optionally add more fields
active_fields.paths.extend(["id", "filepath"])
# Set and save the config
dataset.app_config.active_fields = active_fields
dataset.save()
session.refresh()

In [14]:
active_fields

<ActiveFields: {'exclude': None, 'paths': ['ground_truth', 'predictions', 'id', 'filepath']}>

In [18]:
dataset.default_classes

['healthy', 'damaged', 'dead', 'vcut']